# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1c70dd4250>
├── 'a' --> tensor([[-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150]])
└── 'x' --> <FastTreeValue 0x7f1c70dd4160>
    └── 'c' --> tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                        [ 3.0833, -0.0444,  1.1010, -0.4820],
                        [ 0.9603,  0.6458, -0.2244,  0.9467]])

In [4]:
t.a

tensor([[-0.6229,  0.3926,  1.5458],
        [-3.1575, -0.4036,  0.6150]])

In [5]:
%timeit t.a

75.7 ns ± 0.905 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1c70dd4250>
├── 'a' --> tensor([[-1.4856,  0.8893,  0.3246],
│                   [ 0.0427,  0.5638,  1.3031]])
└── 'x' --> <FastTreeValue 0x7f1c70dd4160>
    └── 'c' --> tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                        [ 3.0833, -0.0444,  1.1010, -0.4820],
                        [ 0.9603,  0.6458, -0.2244,  0.9467]])

In [7]:
%timeit t.a = new_value

75.6 ns ± 1.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150]]),
    x: Batch(
           c: tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                      [ 3.0833, -0.0444,  1.1010, -0.4820],
                      [ 0.9603,  0.6458, -0.2244,  0.9467]]),
       ),
)

In [10]:
b.a

tensor([[-0.6229,  0.3926,  1.5458],
        [-3.1575, -0.4036,  0.6150]])

In [11]:
%timeit b.a

69.8 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0871, -0.7502, -0.9709],
               [ 1.8815,  1.5776,  1.0329]]),
    x: Batch(
           c: tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                      [ 3.0833, -0.0444,  1.1010, -0.4820],
                      [ 0.9603,  0.6458, -0.2244,  0.9467]]),
       ),
)

In [13]:
%timeit b.a = new_value

674 ns ± 5.16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.11 µs ± 9.13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

15.6 µs ± 121 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

288 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

270 µs ± 8.17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1c482c9070>
├── 'a' --> tensor([[[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]],
│           
│                   [[-0.6229,  0.3926,  1.5458],
│                    [-3.1575, -0.4036,  0.6150]]])
└── 'x' --> <FastTreeValue 0x7f1c70de15e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

50.4 µs ± 512 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1c70de1af0>
├── 'a' --> tensor([[-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150],
│                   [-0.6229,  0.3926,  1.5458],
│                   [-3.1575, -0.4036,  0.6150]])
└── 'x' --> <FastTreeValue 0x7f1c70de1e50>
    └── 'c' --> tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                        [ 3.0833, -0.0444,  1.1010, -0.4820],
                 

In [23]:
%timeit t_cat(trees)

46.1 µs ± 1.3 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

88.7 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]],
       
               [[-0.6229,  0.3926,  1.5458],
                [-3.1575, -0.4036,  0.6150]]]),
    x: Batch(
           c: tensor([[[-1.3074,  0.4224,  0.0233, -0.3198],
                       [ 3.0833, -0.0444,  1.1010, -0.4820],
                       [ 0.9603,  0.6458, -0.2244,  0.9467]],
         

In [26]:
%timeit Batch.stack(batches)

111 µs ± 964 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150],
               [-0.6229,  0.3926,  1.5458],
               [-3.1575, -0.4036,  0.6150]]),
    x: Batch(
           c: tensor([[-1.3074,  0.4224,  0.0233, -0.3198],
                      [ 3.0833, -0.0444,  1.1010, -0.4820],
                      [ 0.9603,  0.6458, -0.2244,  0.9467],
                      [-1.3074,  0.4224,  0.0233, -0.3198],
                      [ 3.0833, -

In [28]:
%timeit Batch.cat(batches)

200 µs ± 2.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

624 µs ± 14.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
